# Récupérer des données issues d'un module Arduino dans un programme Python

On se propose ici de voir comment récupérer les données envoyées par un module Arduino au moniteur série directement dans un programme Python. Nous allons explorer plusieurs possibilités :
- récupérer toutes les données envoyées puis les traiter ensuite (tracé de graphes par exemple). On peut aussi vouloir créer un fichier dans lequel stocker ces données, par exemple un fichier au format `.csv` ;
- récupérer les données et les traiter au fur et à mesure où elle sont envoyées.

## Récupérer toutes les données envoyées puis les traiter ensuite

**Le contexte :** On se donne pour objectif de suivre l'évolution de la température d'une salle au cours du temps. L'acquisition peut donc potentiellement se faire sur un temps très long, et l'on souhaite donc avoir la possibilité de stocker les données dans un fichier pour les conserver. Le module Arduino doit donc envoyer périodiquement le couple temps/température, qui doit être intercepté par le programme Python.

### Communication avec le moniteur série

La communication avec le moniteur série peut être réalisée avec la bibliothèque `pyserial` qui s'importe sous le nom `serial`.

> Si vous utilisez ce notebook depuis un fichier présent sur votre PC, tout devrait bien se dérouler (dans la barre d'adresse vous devriez voir `localhost:`). Si vous l'exécutez depuis un serveur distant (si cette page est une publication sur Internet, par exemple sur un serveur généré à l'aide de l'outil Binder), la communication avec le port série de votre PC, *local*, ne se fera pas. Il faut donc veiller à télécharger ce notebook **localement** sur votre PC, en récupérant tous les fichiers [ici](https://github.com/peleroy/formationnumeriqueetpc) puis en les ouvrant avec le logiciel Jupyter Notebook (présent par exemple dans la distribution Winpython sous Windows).
>> Dans la distribution Winpython, sous Windows, cherchez le fichier `Jupyter Notebook.exe` puis exécutez-le.

> La ligne suivante ne doit alors pas retourner d'erreur... !

In [7]:
import serial # Pour la liaison avec le moniteur série du module Arduino

On commence par ouvrir le port série, dont il faut connaître le nom. On peut le trouver ici :

<!---
![PortCom](portcom.png)
--->

<img src="portcom.png" width="400"/>

> Sur cet exemple (sur un OS Ubuntu), le nom du port est `/src/ttyACM0`.

Dans la ligne suivante vous devez ainsi modifier le nom du port, actuellement `COM3`, avec le nom trouvé ci-dessus. Le débit de la liaison étant fixé habituellement à 9600 bauds, cela donne :

In [10]:
# Ouverture du port série
ser=serial.Serial('COM3',9600)

Si vous n'avez aucun message d'erreur, c'est que la communication a pu s'initier. La lecture des données peut ensuite se faire par ligne entière à l'aide de la fonction `readline()`, mais voyons directement le code Python complet.

> Si vous l'exécutez il ne se passera rien puisqu'aucune donnée n'est envoyée sur le moniteur série... C'est la prochaine étape !

In [ ]:
import matplotlib.pyplot as plt # Pour le tracé des graphes
import serial # Pour la liaison avec le module Arduino

# Caractéristiques de l'acquisition
duree=60 # Durée de l'acquisition, en s
intervalle=2000 # Intervalle entre deux mesures, en ms, DOIT être identique dans le programme Arduino
nb_points=int(duree*1000/intervalle) # Estimation du nombre de points nécessaires

# Nom des fichiers de sauvegarde
nomfichier='recuperer_donnees_arduino_temperature'

# Ouverture du port série
ser=serial.Serial('COM3',9600)

# Acquisition
liste_t=[] # Création de la liste, vide
liste_T=[] # Création de la liste, vide

while len(liste_T)<nb_points:
    try:
        s=ser.readline().decode('utf8').split(' ') # Lecture de la ligne, décodage puis séparation des deux valeurs
        liste_t.append(int(s[0])) # Remplissage de la liste des temps
        liste_T.append(float(s[1])) # Remplissage de la liste des températures
        print(int(s[0]),'ms',float(s[1]),'°C') # Affichage des valeurs dans la console (pour vérification)
    except:
        pass

# Fermeture du port série
ser.close()

# Repositionnement de l'origine des temps à t=0 s, et conversion les valeurs en seconde
orig=liste_t[0] # La première valeur sera à soustraire à toutes les autres

for i in range(len(liste_t)):
    liste_t[i]=(liste_t[i]-orig)/1000

# Sauvegarde des données dans un fichier extérieur (optionnel)
fichier=open(nomfichier+'.csv','w')
fichier.write('t(s)'+';'+'T (°C)'+'\n') # Ecriture de la première ligne

for i in range(len(liste_T)):
    fichier.write(str(liste_t[i]).replace('.',',')+';'+str(liste_T[i]).replace('.',',')+'\n') # Ecriture dans le fichier (on remplace le séparateur décimal au passage)

fichier.close() # Fermeture du fichier de sauvegarde des données

# Représentation graphique des points expérimentaux
plt.figure(1)
plt.plot(liste_t,liste_T,'r+')
plt.xlabel('t (s)')
plt.ylabel('Température (°C)')
plt.title('T(t)')
plt.savefig(nomfichier+'.png') # Sauvegarde de la figure au passage
plt.show()


Quelques commentaires sont peut-être nécessaires. La lecture des données se fait avec les lignes suivantes :

```
while len(liste_T)<nb_points:
    try:
        s=ser.readline().decode('utf8').split(' ')
        liste_t.append(int(s[0]))
        liste_T.append(float(s[1]))
        print(int(s[0]),'ms',float(s[1]),'°C')
    except:
        pass
```

La boucle `while` s'arrête lorsque la taille de la liste des températures et égale au nombre de points définis en début de programme. L'utilisation de la structure `try:/except:` n'est pas indispensable, mais elle permet, si une erreur est retournée, de ne pas interrompre le programme et de passer à l'acquisition suivante.

> Sur des expériences de plusieurs heures, rien de plus agaçant que de voir son expérience interrompue pour un bug aléatoire...

La fonction `readline()` permet de lire par ligne entière les données envoyées. On effectue au passage une normalisation de l'encodage en `UTF8`, et on découpe à l'aide de la fonction `split()` la ligne reçue en deux parties, en précisant que la découpe se fait au niveau de l'espace `' '` laissé entre les données du **temps** et de la **température**. Les données reçues, sous forme de chaînes de caractères, doivent être converties en entier pour le temps et en nombre à virgule pour la température.

Enfin on réalise la fermeture de la communication avec le port série.

```
# Fermeture du port série
ser.close()
```

On réalise ensuite un décalage des temps pour que la première valeur corresponde à l'instant $t=0$. On pourrait alors simplement passer à la représentation graphique (fin du programme) si l'on ne souhaite pas sauvegarder les données reçues.

### Sauvegarde des données dans un fichier

Si par contre l'on souhaite sauvegarder les données reçues, on peut écrire celle-ci dans un fichier, par exemple au format `.csv`. C'est l'objet des lignes suivantes :

```
# Sauvegarde des données dans un fichier extérieur (optionnel)
fichier=open(nomfichier+'.csv','w')
fichier.write('t(s)'+';'+'T (°C)'+'\n')

for i in range(len(liste_T)):
    fichier.write(str(liste_t[i]).replace('.',',')+';'+str(liste_T[i]).replace('.',',')+'\n')

fichier.close()
```

**Quelques commentaires :** on crée le fichier avec la fonction `open()` qui prend en arguments le nom du fichier (qui se termine avec l'extension `.csv`) et le type d'interaction (`w` pour *write*). On utilise ensuite la fonction `write()`pour écrire la première ligne et les suivantes, les données étant (c'est un choix arbitraire) séparées par le symbole `;`. On doit écrite les données sous forme de chaînes de caractères avec la fonction `str()` et on remplace au passage le séparateur décimal utilisé en Python `.` par celui utilisé dans les tableurs `,`. On termine par la fermeture du fichier qui réalise aussi sa sauvegarde.

> Le symbole `\n` ajouté en fin de ligne permet de signifier le passage à la ligne.

> Sur des expériences de longue durée, il sera préférable de réaliser l'écriture des données dans un fichier *pendant* l'acquisition, pour qu'en cas d'échec les premières données mesurées soient bien sauvegardées.

### Côté Arduino !

Maintenant il faut faire envoyer les couples temps/température du module Arduino vers le moniteur série, les données étant en lignes et séparées par un espace. Pour plus de commodité on peut aussi utiliser un écran LCD, mais ce n'est pas indispensable :

<img src="suivi_temperature.jpg" width="600"/>

Dans ce montage, un shield Grove est utilisé, sur lequel on a placé un capteur de température Grove sur le port `A0`. L'écran LCD (facultatif) est lui branché sur le port `I2C`).

Le programme Arduino est donné ci-dessous (et téléchargeable [ici](https://github.com/peleroy/formationnumeriqueetpc)), avec sa version Ardublock :

<img src="suivi_temperature_abp.png" width="900"/>

**Quelques commentaires :** Tout ce qui concerne l'écran LCD est facultatif. On se contente de faire la mesure de la température, de l'affecter dans la variable `T`, de mesurer le temps depuis la dernière initialisation du module Arduino, de l'affecter dans la variable `t`, puis de transmettre sur le port série ces deux valeurs, séparées par un espace, avec un retour à la ligne. On ajoute un délai d'attente à l'aide de la fonction `delay()`.

> La valeur de cette pause entre deux mesures doit être le même que celui entré dans la variable `intervalle` dans le programme Arduino. Cela permet d'évaluer le nombre de points à afficher (donc de mesures à effectuer).
>> Vous avez remarqué que le temps est géré côté module Arduino et non pas côté programme Python. C'est voulu, faire le contraire pose divers problèmes. Par exemple faire des pauses dans le programme Python n'affecte que le temps entre deux récupérations des données qui ont en réalité *déjà* transité sur le moniteur série, et toujours stockée dans une sorte de mémoire dynamique appelée << buffer >>, mais là il faudrait plus de temps pour expliquer tout cela... 

### Mise en oeuvre

Il ne reste plus qu'à téléverser le programme sur le module Arduino, puis à lancer le programme Python, reproduit ci-dessous :
> N'oubliez pas de modifier le nom du port série :)

In [ ]:
import matplotlib.pyplot as plt # Pour le tracé des graphes
import serial # Pour la liaison avec le module Arduino

# Caractéristiques de l'acquisition
duree=60 # Durée de l'acquisition, en s
intervalle=2000 # Intervalle entre deux mesures, en ms, DOIT être identique dans le programme Arduino
nb_points=int(duree*1000/intervalle) # Estimation du nombre de points nécessaires

# Nom des fichiers de sauvegarde
nomfichier='recuperer_donnees_arduino_temperature'

# Ouverture du port série
ser=serial.Serial('COM3',9600)

# Acquisition
liste_t=[] # Création de la liste, vide
liste_T=[] # Création de la liste, vide

while len(liste_T)<nb_points:
    try:
        s=ser.readline().decode('utf8').split(' ') # Lecture de la ligne, décodage puis séparation des deux valeurs
        liste_t.append(int(s[0])) # Remplissage de la liste des temps
        liste_T.append(float(s[1])) # Remplissage de la liste des températures
        print(int(s[0]),'ms',float(s[1]),'°C') # Affichage des valeurs dans la console (pour vérification)
    except:
        pass

# Fermeture du port série
ser.close()

# Repositionnement de l'origine des temps à t=0 s, et conversion les valeurs en seconde
orig=liste_t[0] # La première valeur sera à soustraire à toutes les autres

for i in range(len(liste_t)):
    liste_t[i]=(liste_t[i]-orig)/1000

# Sauvegarde des données dans un fichier extérieur (optionnel)
fichier=open(nomfichier+'.csv','w')
fichier.write('t(s)'+';'+'T (°C)'+'\n') # Ecriture de la première ligne

for i in range(len(liste_T)):
    fichier.write(str(liste_t[i]).replace('.',',')+';'+str(liste_T[i]).replace('.',',')+'\n') # Ecriture dans le fichier (on remplace le séparateur décimal au passage)

fichier.close() # Fermeture du fichier de sauvegarde des données

# Représentation graphique des points expérimentaux
plt.figure(1)
plt.plot(liste_t,liste_T,'r+')
plt.xlabel('t (s)')
plt.ylabel('Température (°C)')
plt.title('T(t)')
plt.savefig(nomfichier+'.png') # Sauvegarde de la figure au passage
plt.show()

Les données devraient s'afficher au fur et à mesure qu'elles sont collectées, puis le graphe final. Le fichier `.csv` de sauvegarde devrait faire son apparition dans le même répertoire, ainsi qu'un fichier image du graphe.

L'acquisition s'effectue ici sur $60$ s avec une mesure toutes les $2$ s, mais ceci est bien sûr réglable :
- dans le programme Python par l'intermédiaire des variables `duree` et `intervalle` ;
- dans le programme Arduino la valeur de la fonction `delay()` doit être identique à la valeur affectée à la variable `intervalle` dans le programme Python pour que celui-ci puisse estimer le nombre de points de mesure à effectuer.

Ci-dessous le résultat d'un enregistrement dans une pièce de vie pendant $9$ h, toutes les $200$ s :

<img src="temperature_piece.png" width="600"/>


# Récupérer les données et les traiter au fur et à mesure où elle sont envoyées

Ou comment essayer de créer une petite interface d'acquisition...

> Il n'est pas question ici d'essayer de montrer qu'un module Arduino peut remplacer une interface d'acquisition, il ne le peut pas complètement, et n'a pas vocation à le faire. L'objectif n'est pas de remplacer nos interfaces d'acquisition, performantes, par un autre système - moins performant - mais bien d'utiliser ses possibilités pour faire de la Physique-Chimie plus ouverte, c'est à dire :
> - de s'interroger sur le fonctionnement des boîtes noires qu'il nous arrive d'utiliser dans notre activité de scientifique (typiquement nos interfaces d'acquisition) ;
> - d'ouvrir le champ des possibles en termes d'expérimentations, notamment d'expérimentations hors laboratoire, réalisables simplement et même chez soi (la culture du DIY) ;
> - d'apporter cette culture du développement d'outils simples, maîtrisés car faciles à mettre en oeuvre, pour répondre à une problématique scientifique, et qui envahit aujourd'hui le monde de la recherche et de l'industrie ;
> - et bien d'autres choses encore...
>> Ouvrir des boîtes noires, c'est un peu notre raison d'être non :) ?

**Le contexte :** On se propose de réaliser un petit sonar pour effectuer de la cartographie de relief. On devra pouvoir voir à l'écran se dessiner les courbes du relief, au même instant que la mesure s'effectue.

## Première solution sans programme Python : le moniteur série

Il existe un outil interne inclus dans l'IDE Arduino appelé << traceur série >>. Il permet (notamment) de générer un graphe représentant l'évolution d'une grandeur envoyée sur le moniteur série en fonction du temps. Ce graphe est *glissant*, c'est à dire que les données ne sont pas gardées en mémoire mais *défilent* au cours du temps.

On va commencer par envoyer sur le moniteur série la distance retournée par un module télémètre Grove. Le montage et le programme, ci-dessous, ne posent aucune difficulté.

<img src="telemetre.jpg" width="600"/>

<img src="telemetre_abp.png" width="900"/>

Le traceur série se sélectionne alors ainsi :

<img src="traceur_serie.png" width="400"/>

Et l'on obtient un petit sonar, par exemple au dessus d'une table avec divers objets :

<img src="relief_table.png" width="800"/>


## Deuxième solution avec un programme Python

L'inconvénient du traceur série est qu'il ne permet pas de stocker les résultats. On va donc simplement déjà dans un premier temps régler ce souci, c'est à dire récupérer les données à l'aide d'un programme Python, puis les enregistrer pour les conserver. Dans un second temps on fera le tout *à la volée*.

Du côté Arduino d'abord, on va envoyer deux informations sur le moniteur série : la distance en cm et le temps en s. Cela donne assez simplement :

<img src="sonar_abp.png" width="900"/>

### Sauvegarder les données

Le programme Python suivant réalise ensuite le travail recherché, il ne devrait pas poser trop de soucis de compréhension maintenant :
> N'oubliez pas de modifier le nom du port série :)

In [ ]:
import matplotlib.pyplot as plt
import serial


# Caractéristiques de l'acquisition
duree=10000 # (en ms)
nomfichier='recuperer_donnees_arduino_sonar_python'

# Ouverture du port de communication
ser=serial.Serial('COM3',9600)

# Pour provoquer une réinitialisation au cas où le port n'est pas fermé (vidage du buffer)
ser.close()
ser.open()

# Acquisition
liste_d=[]
liste_t=[]
t=0
while t<duree:
    try:
        s=ser.readline().decode('utf8').split(' ')
        d=int(s[0])
        t=int(s[1])
        liste_d.append(d)
        liste_t.append(t)
        print(d,t) # Affichage de la valeur et du temps correspondant dans la console
    except:
        t=0 # Pour éviter de sortir de la boucle si une erreur survient suite à l'affectation dans la variable 't'

# Fermeture du port de communication
ser.close()

# Sauvegarde des données dans un fichier extérieur (optionnel)
fichier=open(nomfichier+'.csv','w')
fichier.write('t(s)'+';'+'d (m)'+'\n') # Ecriture de la première ligne

for i in range(len(liste_d)):
    fichier.write(str(liste_t[i]).replace('.',',')+';'+str(liste_d[i]).replace('.',',')+'\n') # Ecriture dans le fichier (on remplace le séparateur décimal au passage)

fichier.close() # Fermeture du fichier de sauvegarde des données

# Représentation graphique et sauvegarde de celle-ci
plt.plot(liste_t,liste_d,'r-')
plt.xlabel('t (ms)')
plt.ylabel('d (cm)')
plt.savefig(nomfichier+'.png')
plt.show()


Le résultat est donné ci-dessous :

<img src="relief_table_python.png" width="600"/>

### À la volée !

L'idée << naïve >> (que j'ai eue la première fois) est tout simplement de faire une boucle et de retracer le graphe à chaque fois. Malheureusement cela ne fonctionne pas, les temps de traitement pour des opérations graphiques de la bibliothèque `matplotlib` sont trop longs.
> Une solution avait été développée avec les graphes interactifs (fonctions `ion()`et `ioff()`) mais elle ne doit plus être utilisée, au profit des fonctions d'animation qui ont été développées dans le but de réaliser rapidement ces opérations graphiques.

Partons de la solution que je vous propose :

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
import serial


# Caractéristiques de l'acquisition
duree=10000 # (en ms)
dmax=210 # Distance maximale à afficher (cm)

# Représentation graphique
fig=plt.figure(1)
plt.xlabel('t (ms)')
plt.ylabel('d (cm)')
plt.xlim(0,duree)
plt.ylim(0,dmax)

line,=plt.plot([],[],'r-')

# Ouverture du port de communication
#ser=serial.Serial('COM3',9600)
ser=serial.Serial('/dev/ttyACM1',9600)

# Il semble nécessaire d'effectuer un vidage du buffer au préalable... ?
ser.close()
ser.open()

# Initialisation des listes
liste_d=[]
liste_t=[]
t0=0

def mesure():
    global liste_d # On doit préciser que ces variables sont globales (cf. partie réinitialisation du graphe)
    global liste_t
    global t0
    
    try:
        ser.open() # Il semble nécessaire d'ouvrir le port lors du premier appel de la fonction... ?
    except:
        pass
    
    try:
        s=ser.readline().decode('utf8').split(' ')
        d=int(s[0])
        t=int(s[1])
        liste_d.append(d)
        liste_t.append(t-t0)
        
        if t-t0>duree: # Réinitialisation de l'affichage en bout de graphe
            t0=t
            liste_d=[d]
            liste_t=[0]
    except:
        pass
    
    return(liste_t,liste_d)

def animate(i):
    line.set_data(mesure())
    return(line,)

anim=animation.FuncAnimation(fig,animate,frames=1,interval=0,blit=True)
plt.show()

# Fermeture du port de communication
ser.close()


**Quelques commentaires :** Le fonctionnement peut s'interpréter ainsi : création d'un graphe vide avec `line,=plt.plot([],[],'r-')` puis définition de deux fonction, l'une `mesure()` dans laquelle on effectue les acquisitions tout à fait normalement, et qui retourne les deux listes mises à jour des valeurs ; et l'autre `animate()` qui remplit le graphe vide avec ces listes. On appelle ensuite la fonction d'animation et on génère son affichage avec `plt.plot()`.
> Vous remarquerez que le graphe se régénère à la fin de la fenêtre d'acquisition. Si l'on ne veut pas de cette fonction, il suffit de remplacer la condition de réinitialisation :
>
> ```
> if t-t0>duree: # Réinitialisation de l'affichage en bout de graphe
      t0=t
      liste_d=[d]
      liste_t=[0]
> ```
>
> par une condition d'interruption :
>
> ```
> if t-t0>duree: # Fin du processus en bout de graphe
      anim.event_source.stop()
> ```

> On peut aussi modifier la durée d'acquisition (durée d'un balayage) avec la variable `duree`.

Pour réaliser une sauvegarde des données, par exemple à la fin d'un balayage, il suffit de remplacer la condition de réinitialisation par une sauvegarde des listes de données :

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import animation
import serial


# Caractéristiques de l'acquisition
duree=10000 # (en ms)
dmax=210 # Distance maximale à afficher (cm)
nomfichier='recuperer_donnees_arduino_sonar_a_la_volee_sauvegardes'

# Représentation graphique
fig=plt.figure(1)
plt.xlabel('t (ms)')
plt.ylabel('d (cm)')

plt.xlim(0,duree+1000)
plt.ylim(0,dmax)

line,=plt.plot([],[],'r-')

# Ouverture du port de communication
#ser=serial.Serial('COM3',9600)
ser=serial.Serial('/dev/ttyACM1',9600)

# Pour provoquer une réinitialisation au cas où le port n'est pas fermé (vidage du buffer)
ser.close()
ser.open()

# Initialisation des listes
liste_d=[]
liste_t=[]
t0=0

def mesure():
    global liste_d # On doit préciser que ces variables sont globales (cf. partie réinitialisation du graphe)
    global liste_t
    global t0
    
    try:
        ser.open()
    except:
        pass
    
    try:
        s=ser.readline().decode('utf8').split(' ')
        d=int(s[0])
        t=int(s[1])
        liste_d.append(d)
        liste_t.append(t-t0)
        
        if t-t0>duree: # Fin du processus
            # Sauvegarde des données au préalable
            fichier=open(nomfichier+'.csv','w')
            
            fichier.write('t(s)'+';'+'d (m)'+'\n') # Ecriture de la première ligne
            for i in range(len(liste_d)):
                fichier.write(str(liste_t[i]).replace('.',',')+';'+str(liste_d[i]).replace('.',',')+'\n')
            
            fichier.close() # Fermeture du fichier de sauvegarde des données
            
            # Fin du processus
            anim.event_source.stop()
    except:
        pass
    
    return(liste_t,liste_d)

def animate(i):
    line.set_data(mesure())
    return(line,)

anim=animation.FuncAnimation(fig,animate,frames=1,interval=0,blit=True)
plt.show()

# Fermeture du port de communication
ser.close()


On obtient le même type de résultat que précédemment :

<img src="relief_table_python.png" width="600"/>

Et voilà... !
<div style="text-align:right;">Pierre-Emmanuel LEROY</div>